In [1]:
#%%capture
# remove all these logs
import logging
#logger = logging.getLogger("mechanize")
# only log really bad events
#logger.setLevel(logging.ERROR)

import sys
sys.path.append("../")
from mwdata.sensitive_data.sensitive_data import *
from faker import Faker
import pandas as pd


[2020-06-14 18:50:02,449][presidio][INFO]nlp_engine not provided. Creating new SpacyNlpEngine instance
[2020-06-14 18:50:02,774][presidio][INFO]Loading NLP model: spaCy en_core_web_lg

===================== Info about model 'en_core_web_lg' =====================

lang             en                            
name             core_web_lg                   
license          MIT                           
author           Explosion                     
url              https://explosion.ai          
email            contact@explosion.ai          
description      English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.
sources          [{'name': 'OntoNotes 5', 'url': 'https://catalog.ldc.upenn.edu/LDC2013T19', 'license': 'commercial (licensed by Explosion)'}, {'name': 'Common Crawl'}]
pipeline         ['tagger', 'parser', 'ner']   
version          2.2.5  

# Create fake data 

In [2]:
fake = Faker()
Faker.seed(0)
df = pd.DataFrame([fake.profile() for i in range(10)])
df.head(2)

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Secretary/administrator,"Fisher, Green and Dixon",415-39-7809,24219 Archer Mountain Suite 924\nNorth Melissa...,"(73.9837235, 163.824695)",O-,"[http://sellers.com/, https://garrett.com/, ht...",sandraharris,Doris Martinez,F,"19659 Ivan Stravenue Apt. 471\nLake Nancyside,...",mary84@yahoo.com,1936-07-02
1,Senior tax professional/tax inspector,"Lawrence, Herring and Riley",462-64-5856,"5330 Wilson Fields Suite 560\nEast Heiditown, ...","(71.6439095, -140.273217)",A+,"[https://wood-hooper.com/, http://martinez.net/]",jeffreylucas,Jeffery Garcia,M,"0916 Michael Row\nSellersville, WI 08109",imoore@yahoo.com,1920-05-27


# Redact sensitive data

In [3]:
sensitive_data(df, redact=True, score_threshold=0.50).head(2)

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Secretary/administrator,"<PERSON>, <PERSON> and <PERSON>",<US_SSN>,24219 <LOCATION> Suite 924\nNorth Melissaborou...,"(Decimal('73.9837235'), Decimal('163.824695'))",O-,"['http://<DOMAIN_NAME>/', 'https://<DOMAIN_NAM...",sandraharris,<PERSON>,F,"<DATE_TIME> <PERSON> Apt. 471\nLake Nancyside,...",<EMAIL_ADDRESS>,<DATE_TIME>
1,Senior tax professional/tax inspector,"<PERSON>, <PERSON> and <PERSON>",<US_SSN>,"5330 Wilson Fields Suite 560\n<LOCATION>, <LOC...","(Decimal('71.6439095'), Decimal('-140.273217'))",A+,"['https://<DOMAIN_NAME>/', 'http://<DOMAIN_NAM...",jeffreylucas,<PERSON>,M,"<DATE_TIME>Sellersville, <LOCATION> 08109",<EMAIL_ADDRESS>,<DATE_TIME>


# Redact sensitive data using selected columns

In [4]:
cols = ['birthdate', 'mail']
df[cols] = sensitive_data(df, redact=True, score_threshold=0.50, cols=['birthdate', 'mail'])
df.head(2)

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Secretary/administrator,"Fisher, Green and Dixon",415-39-7809,24219 Archer Mountain Suite 924\nNorth Melissa...,"(73.9837235, 163.824695)",O-,"[http://sellers.com/, https://garrett.com/, ht...",sandraharris,Doris Martinez,F,"19659 Ivan Stravenue Apt. 471\nLake Nancyside,...",<EMAIL_ADDRESS>,<DATE_TIME>
1,Senior tax professional/tax inspector,"Lawrence, Herring and Riley",462-64-5856,"5330 Wilson Fields Suite 560\nEast Heiditown, ...","(71.6439095, -140.273217)",A+,"[https://wood-hooper.com/, http://martinez.net/]",jeffreylucas,Jeffery Garcia,M,"0916 Michael Row\nSellersville, WI 08109",<EMAIL_ADDRESS>,<DATE_TIME>


In [5]:
df['birthdate'].value_counts()

<DATE_TIME>    10
Name: birthdate, dtype: int64

In [6]:
df['mail'].value_counts()

<EMAIL_ADDRESS>    10
Name: mail, dtype: int64

# Identify infotypes in each column

In [7]:
fake = Faker()
df = pd.DataFrame([fake.profile() for i in range(10)])
df.head(2)

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,"Copywriter, advertising",Bentley Group,051-50-0546,"02394 Bush Mission\nPort Michael, NY 68150","(88.678598, 168.167349)",O-,"[https://www.lewis.com/, http://buchanan.org/,...",chambersjill,Kelly Lopez,F,"53048 Steven Route Apt. 677\nNorth John, DE 28480",chrissimmons@hotmail.com,2017-04-28
1,Fine artist,"Wright, Robinson and Pittman",286-46-5237,Unit 1849 Box 8347\nDPO AA 61066,"(53.9257025, 160.567443)",B+,"[http://thompson.net/, https://www.patterson-w...",julian31,Ms. Crystal Moran,F,"74922 Tran Heights Suite 026\nEast Vickieland,...",heather33@hotmail.com,2015-07-14


In [8]:
redact_df = sensitive_data(df, redact=True)
redact_df.head(2)

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,"Copywriter, advertising",Bentley Group,<US_SSN>,"02394 <PERSON> Mission\nPort <LOCATION>, <LOCA...","(Decimal('88.678598'), Decimal('168.167349'))",O-,"['https://<DOMAIN_NAME>/', 'http://<DOMAIN_NAM...",chambersjill,<PERSON>,F,"53048 Steven Route Apt. 677\nNorth John, DE 28480",<EMAIL_ADDRESS>,<DATE_TIME>
1,Fine artist,<PERSON> and <PERSON>,<US_SSN>,Unit 1849 Box 8347\nDPO AA 61066,"(Decimal('53.9257025'), Decimal('160.567443'))",B+,"['http://<DOMAIN_NAME>/', 'https://<DOMAIN_NAM...",julian31,Ms. <PERSON>,F,<DATE_TIME> Tran Heights Suite 026\n<LOCATION>...,<EMAIL_ADDRESS>,2015-07-14


# Modin

In [9]:
# import os
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
# import modin.pandas as mpd
# from faker import Faker
# import pandas as pd

In [16]:
# fake = Faker()
# Faker.seed(0)
# df = pd.DataFrame([fake.profile() for i in range(100)])
# df.head(2)

In [12]:
# fake = Faker()
# df = mpd.DataFrame([fake.profile() for i in range(100)])
# df.head(2)

In [17]:
#type(df)

In [18]:
# %%time
# df['myname'] = 'richard'
# x = encrypt_data(df)

In [19]:
# df = mpd.DataFrame(["John Doe", "John Doe"])
# df

In [20]:
#df.iloc[1,0]

In [21]:
#x['myname'].value_counts()